## Efficient computations of the Iwasawa $\lambda$-invariant $\lambda_p(\chi)$ for arbitary Dirichlet characters  $\chi$ using characters $\psi$ of order $p^n$.

## Alternative approach: use $L_p(1,\chi)$ and  $L_p(1,\chi \psi)$.

### Preprint: *Special values of $p$-adic $L$-functions and Iwasawa λ-invariants of Dirichlet characters* by Heiko Knospe.

In [1]:
def lambda_lp1(chi,p):
    r"""
    Returns the lambda-invariant of the Iwasawa power series of $\chi$,
    where $\chi$ is a character of the first kind.
    
    First, it is checked whether $\lambda=0$. This can be done very fast using $L_p(1,\chi)$.
    If $\lambda>0$ then $\chi$ is multiplied with a character $\psi$ of order $p$ of the second kind. Then $L_p(1,\chi \psi)$ is
    used to compute $\lambda$-invariants $\leq p-1$ of the original character $\chi$. If $\lambda \geq p$ we multiply with a character
    $\psi$ of higher $p$-th power order. This takes more time as the conductor of $\chi \psi$ increases. However, 
    large $\lambda$-invariants are very rare.
    
    INPUT:
    
  
    - ``chi``-- a Dirichlet character, uses the SageMath class DirichletCharacter.
    
    - ``p`` -- an odd prime number.
    
    
    OUTPUT: lambda-invariant
    
    
    """
    if p==2 or p.is_prime() == False:
        print("p must be an odd prime number.")
        return()
    
    if chi.is_primitive() == False:
        print("chi must be a primitive character.")
        return()
        
    if (chi.is_odd()): 
        # non-trivial p-adic L-function requires even character 
        print("The p-adic L-function is zero.")
        return()

    cond = chi.conductor() # conductor of $\chi$
    
    if gcd(cond,p*p)==p*p:
        print("chi must be a character of the first kind.")
        return() 
              
    ord = chi.order() # order of $\chi$
    
    if gcd(ord,p)>1:
        print("Not yet implemented.")
        return()


    sum=0
    K=Qp(p,5)
    d=lcm(cond,p)
    
    F=CyclotomicField(ord*p)
    v=QQ.valuation(p).extensions(F)[0]

    # get character psi of order p and conductor p^2
    H = DirichletGroup(p*p, base_ring=CyclotomicField(p))
    for i in range(H.order()):
        if H[i].is_primitive() and H[i].order()==p:
            psi=H[i]
        exit



    # compute L_p(1,\chi)
    for a in range(d):
        if gcd(a,p)==1:
            ap=K(a)
            sum = sum + chi(a)*(log(ap)/d - 1/(2*ap) - d/(12*ap*ap)).lift()
            
    if v(sum)==0:
        return(0)
    else:    
        # case lambda>0
        sum1=sum
        G=DirichletGroup(lcm(d,p*p),base_ring=F)
        prod=G(psi)*G(chi) 
        c=prod.conductor()

        # compute L_p(1,\chi) - L_p(1,\chi * \psi)
        for a in range(c):
            if gcd(a,p)==1:
                ap=K(a)
                sum = sum - prod(a)*(log(ap)/c - 1/(2*ap) - c/(12*ap*ap)).lift()
        lam=v(sum)*(p-1)
        if lam <= p-1:
            return(lam)
        else:
            # case lambda>p-1
            
            # get character psi of order p^2 and conductor p^3
            H = DirichletGroup(p*p*p, base_ring=CyclotomicField(p*p))
            for i in range(H.order()):
                if H[i].is_primitive() and H[i].order()==p*p:
                    psi=H[i]
                    exit
                    
            F=CyclotomicField(ord*p*p)
            v=QQ.valuation(p).extensions(F)[0]  
            G=DirichletGroup(lcm(cond,p*p*p),base_ring=F)
            prod=G(psi)*G(chi) # twist by character psi of order p^2
            c=prod.conductor()

            # compute L_p(1,\chi) - L_p(1,\chi * \psi)
            sum=F(sum1)
            for a in range(c):
                if gcd(a,p)==1:
                    ap=K(a)
                    sum = sum - prod(a)*(log(ap)/c - 1/(2*ap) - c/(12*ap*ap)).lift()
            lam=v(sum)*(p-1)*p
            if lam <= (p-1)*p:                
                return(lam)
            else:
                # case lambda>(p-1)*p
                
                # get character psi of order p^3 and conductor p^4
                H = DirichletGroup(p*p*p*p, base_ring=CyclotomicField(p*p*p))
                for i in range(H.order()):
                    if H[i].is_primitive() and H[i].order()==p*p*p:
                        psi=H[i]
                        exit
                        
                F=CyclotomicField(ord*p*p*p)
                v=QQ.valuation(p).extensions(F)[0]  
                G=DirichletGroup(lcm(cond,p*p*p*p),base_ring=F)
                prod=G(psi)*G(chi) # twist by character psi of order p^3
                c=prod.conductor()

                # compute L_p(1,\chi) - L_p(1,\chi * \psi)
                sum=F(sum1)
                for a in range(c):
                    if gcd(a,p)==1:
                        ap=K(a)
                        sum = sum - prod(a)*(log(ap)/c - 1/(2*ap) - c/(12*ap*ap)).lift()
                lam=v(sum)*(p-1)*p*p
                return(lam) # might be wrong for lambda>(p-1)*p^2, but this occures extremly (!) rarely.

In [2]:
# Example: p=3, 700<conductor<740, order=2, even quadratic characters
# same result as with function lambda_invariant

p=3
print(f'p={p}')

for n in range(701,740):
            H = DirichletGroup(n, base_ring=CyclotomicField(2))
            for i in range(H.order()):
                if H[i].order()==2 and H[i].is_even()==True and H[i].is_primitive() and gcd(H[i].conductor(),p)==1:
                    chi = H[i]
        
                    print(f'{chi} ==> lambda={lambda_lp1(chi,p)}')    

p=3
Dirichlet character modulo 701 of conductor 701 mapping 2 |--> -1 ==> lambda=0
Dirichlet character modulo 709 of conductor 709 mapping 2 |--> -1 ==> lambda=0
Dirichlet character modulo 712 of conductor 712 mapping 535 |--> 1, 357 |--> -1, 537 |--> -1 ==> lambda=0
Dirichlet character modulo 713 of conductor 713 mapping 373 |--> -1, 530 |--> -1 ==> lambda=0
Dirichlet character modulo 716 of conductor 716 mapping 359 |--> -1, 181 |--> -1 ==> lambda=2
Dirichlet character modulo 721 of conductor 721 mapping 619 |--> -1, 211 |--> -1 ==> lambda=0
Dirichlet character modulo 728 of conductor 728 mapping 183 |--> -1, 365 |--> -1, 521 |--> -1, 561 |--> -1 ==> lambda=1
Dirichlet character modulo 733 of conductor 733 mapping 6 |--> -1 ==> lambda=3
Dirichlet character modulo 737 of conductor 737 mapping 68 |--> -1, 672 |--> -1 ==> lambda=0


In [3]:
# Example: p=7, conductor<100, order=3, even cubic characters
# same result as with function lambda_invariant

p=7

print(f'p={p}')

for n in range(2,100):
            H = DirichletGroup(n, base_ring=CyclotomicField(3))
            for i in range(H.order()):
                if H[i].order()==3 and H[i].is_even()==True and H[i].is_primitive() and gcd(H[i].conductor(),p)==1:
                    chi = H[i]
        
                    print(f'{chi} ==> lambda={lambda_lp1(chi,p)}')    

p=7
Dirichlet character modulo 9 of conductor 9 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 9 of conductor 9 mapping 2 |--> -zeta3 - 1 ==> lambda=2
Dirichlet character modulo 13 of conductor 13 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 13 of conductor 13 mapping 2 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 19 of conductor 19 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 19 of conductor 19 mapping 2 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 31 of conductor 31 mapping 3 |--> zeta3 ==> lambda=0
Dirichlet character modulo 31 of conductor 31 mapping 3 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 37 of conductor 37 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 37 of conductor 37 mapping 2 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 43 of conductor 43 mapping 3 |--> zeta3 ==> lambda=0
Dirichlet character modulo 43 of conductor 43 mapping 3 |--> -zeta3 - 1 ==> lambda=0
Dirichlet 

In [4]:
# Example: p=5, conductor<200, order=3, even cubic characters
p=5

print(f'p={p}')

for n in range(2,200):
            H = DirichletGroup(n, base_ring=CyclotomicField(3))
            for i in range(H.order()):
                if H[i].order()==3 and H[i].is_even()==True and H[i].is_primitive() and gcd(H[i].conductor(),p)==1:
                    chi = H[i]
        
                    print(f'{chi} ==> lambda={lambda_lp1(chi,p)}')    

p=5
Dirichlet character modulo 7 of conductor 7 mapping 3 |--> zeta3 ==> lambda=0
Dirichlet character modulo 7 of conductor 7 mapping 3 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 9 of conductor 9 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 9 of conductor 9 mapping 2 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 13 of conductor 13 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 13 of conductor 13 mapping 2 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 19 of conductor 19 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 19 of conductor 19 mapping 2 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 31 of conductor 31 mapping 3 |--> zeta3 ==> lambda=0
Dirichlet character modulo 31 of conductor 31 mapping 3 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 37 of conductor 37 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 37 of conductor 37 mapping 2 |--> -zeta3 - 1 ==> lambda=0
Dirichlet char

In [5]:
# Example of a large lambda-invariant
H = DirichletGroup(71068, base_ring=CyclotomicField(2))
chi=H[7] # even quadratic character

In [6]:
p=3

print(f'p={p}')

print(f'{chi} ==> lambda={lambda_lp1(chi,p)}')  
# takes several minutes. lambda=11, ok!

p=3
Dirichlet character modulo 71068 of conductor 71068 mapping 35535 |--> -1, 33905 |--> -1, 53629 |--> -1 ==> lambda=11
